In [1]:
from __future__ import print_function
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
from keras.utils import plot_model
import numpy as np
import os
import math
import tensorflow as tf
from keras.layers import *



class FractionalPooling2D(Layer):
	def __init__(self, pool_ratio = None, pseudo_random = False, overlap = False, name ='FractionPooling2D', **kwargs):
		self.pool_ratio = pool_ratio
		self.input_spec = [InputSpec(ndim=4)]
		self.pseudo_random = pseudo_random
		self.overlap = overlap
		self.name = name
		super(FractionalPooling2D, self).__init__(**kwargs)
		
	def call(self, input):
		[batch_tensor,row_pooling,col_pooling] = tf.nn.fractional_avg_pool(input, pooling_ratio = self.pool_ratio, pseudo_random = self.pseudo_random, overlapping = self.overlap, seed = 0)
		return(batch_tensor)
		
	def compute_output_shape(self, input_shape):
	
		if(K.common.image_dim_ordering() == 'channels_last' or K.common.image_dim_ordering() == 'tf'):
			if(input_shape[0] != None):
				batch_size = int(input_shape[0]/self.pool_ratio[0])
			else:
				batch_size = input_shape[0]
			width = int(input_shape[1]/self.pool_ratio[1])
			height = int(input_shape[2]/self.pool_ratio[2])
			channels = int(input_shape[3]/self.pool_ratio[3])
			return(batch_size, width, height, channels)
			
		elif(K.image_dim_ordering() == 'channels_first' or K.image_dim_ordering() == 'th'):
			if(input_shape[0] != None):
				batch_size = int(input_shape[0]/self.pool_ratio[0])
			else:
				batch_size = input_shape[0]
			channels = int(input_shape[1]/self.pool_ratio[1])
			width = int(input_shape[2]/self.pool_ratio[2])
			height = int(input_shape[3]/self.pool_ratio[3])
			return(batch_size, channels, width, height)
		
	def get_config(self):
		config = {'pooling_ratio': self.pool_ratio, 'pseudo_random': self.pseudo_random, 'overlap': self.overlap, 'name':self.name}
		base_config = super(FractionalPooling2D, self).get_config()
		return dict(list(base_config.items()) + list(config.items()))
		
	def build(self, input_shape):
		self.input_spec = [InputSpec(shape=input_shape)]



# Training parameters
batch_size = 50  # orig paper trained all networks with batch_size=128
epochs = 200
data_augmentation = True
num_classes = 10

# Subtracting pixel mean improves accuracy
subtract_pixel_mean = True

# Model parameter
# ----------------------------------------------------------------------------
#           |      | 200-epoch | Orig Paper| 200-epoch | Orig Paper| sec/epoch
# Model     |  n   | ResNet v1 | ResNet v1 | ResNet v2 | ResNet v2 | GTX1080Ti
#           |v1(v2)| %Accuracy | %Accuracy | %Accuracy | %Accuracy | v1 (v2)
# ----------------------------------------------------------------------------
# ResNet20  | 3 (2)| 92.16     | 91.25     | -----     | -----     | 35 (---)
# ResNet32  | 5(NA)| 92.46     | 92.49     | NA        | NA        | 50 ( NA)
# ResNet44  | 7(NA)| 92.50     | 92.83     | NA        | NA        | 70 ( NA)
# ResNet56  | 9 (6)| 92.71     | 93.03     | 93.01     | NA        | 90 (100)
# ResNet110 |18(12)| 92.65     | 93.39+-.16| 93.15     | 93.63     | 165(180)
# ResNet164 |27(18)| -----     | 94.07     | -----     | 94.54     | ---(---)
# ResNet1001| (111)| -----     | 92.39     | -----     | 95.08+-.14| ---(---)
# ---------------------------------------------------------------------------
n = 18

# Model version
# Orig paper: version = 1 (ResNet v1), Improved ResNet: version = 2 (ResNet v2)
version = 1

# Computed depth from supplied model parameter n
if version == 1:
    depth = n * 6 + 2
elif version == 2:
    depth = n * 9 + 2

# Model name, depth and version
model_type = 'ResNet%dv%d' % (depth, version)

# Load the CIFAR10 data.
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Input image dimensions.
input_shape = x_train.shape[1:]

# Normalize data.
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# If subtract pixel mean is enabled
if subtract_pixel_mean:
    x_train_mean = np.mean(x_train, axis=0)
    x_train -= x_train_mean
    x_test -= x_train_mean

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print('y_train shape:', y_train.shape)

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr


def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder

    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)

    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x


def resnet_v1(input_shape, depth, num_classes=10):
    """ResNet Version 1 Model builder [a]

    Stacks of 2 x (3 x 3) Conv2D-BN-ReLU
    Last ReLU is after the shortcut connection.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filters is
    doubled. Within each stage, the layers have the same number filters and the
    same number of filters.
    Features maps sizes:
    stage 0: 32x32, 16
    stage 1: 16x16, 32
    stage 2:  8x8,  64
    The Number of parameters is approx the same as Table 6 of [a]:
    ResNet20 0.27M
    ResNet32 0.46M
    ResNet44 0.66M
    ResNet56 0.85M
    ResNet110 1.7M

    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (eg 20, 32, 44 in [a])')
    # Start model definition.
    num_filters = 16
    num_res_blocks = int((depth - 2) / 6)
    print('num res blocks:', num_res_blocks)
    act_size = 32.0

    inputs = Input(shape=input_shape, batch_shape=(batch_size, )+input_shape)
    #inputs = Input(shape=input_shape)
    x = resnet_layer(inputs=inputs)
    top = x
    # Instantiate the stack of residual units
    for stack in range(3):
        for res_block in range(num_res_blocks):
            strides = 1
            if stack > 0 and res_block == 0:  # first layer but not first stack
                ratio = 1.23
                x = FractionalPooling2D((1.0,ratio, ratio,1.0), overlap=True)(x)
                act_size = int(act_size / ratio)
                print(act_size)


            if stack > 0 and res_block == 6:  # first layer but not first stack
                ratio = 1.238
                x = FractionalPooling2D((1.0,ratio,ratio,1.0), overlap=True)(x) 
                act_size = int(act_size / ratio)
                print(act_size)


            if stack > 0 and res_block == 12:  # first layer but not first stack
                ratio = 1.25
                x = FractionalPooling2D((1.0,ratio, ratio,1.0), overlap=True)(x)
                act_size = int(act_size / ratio)
                print(act_size)

                print('res now', 32/(2*stack))
                y = MaxPooling2D(2*stack)(top)
                y = Convolution2D(num_filters, (1,1), kernel_initializer='he_normal', kernel_regularizer=l2(1e-4))(y)
                y = BatchNormalization()(y)
                y = Activation('relu')(y)
                x = Add()([x, y])


            y = resnet_layer(inputs=x,
                             num_filters=num_filters,
                             strides=strides)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters,
                             activation=None)
            
            if stack > 0 and res_block == 0:  # first layer but not first stack
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])
            x = Activation('relu')(x)
        num_filters *= 2

    # Add classifier on top.
    # v1 does not use BN after last shortcut connection-ReLU
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model



model = resnet_v1(input_shape=input_shape, depth=depth)

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=lr_schedule(0)),
              metrics=['accuracy'])
model.summary()
print(model_type)
plot_model(model, 'model.png', show_shapes=True)

# Prepare model model saving directory.
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'cifar10_%s_model.{epoch:03d}.h5' % model_type
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [ 
    lr_reducer, 
    lr_scheduler]

# Run training, with or without data augmentation.
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True,
              callbacks=callbacks)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator()

    # Compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    # datagen.fit(x_train)
    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                        validation_data=(x_test, y_test),
                        epochs=epochs, verbose=1, workers=4,
                        callbacks=callbacks)
    # model.fit(x_train, y_train, batch_size=batch_size, callbacks=callbacks, epochs=epochs, verbose=1, workers=4)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Using TensorFlow backend.


x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
y_train shape: (50000, 1)
num res blocks: 18
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
`seed2` and `deterministic` args are deprecated.  Use fractional_avg_pool_v2.
26
21
16
res now 16.0

13
10
8
res now 8.0

Learning rate:  0.001
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (50, 32, 32, 3)      0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (50, 32, 32, 16)     448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization

Using real-time data augmentation.

Epoch 1/200
Learning rate:  0.001
1000/1000 [==============================] - 173s 173ms/step - loss: 2.3683 - accuracy: 0.4606 - val_loss: 2.1565 - val_accuracy: 0.5048
Epoch 2/200
Learning rate:  0.001
1000/1000 [==============================] - 146s 146ms/step - loss: 1.6668 - accuracy: 0.6438 - val_loss: 2.6544 - val_accuracy: 0.4170
Epoch 3/200
Learning rate:  0.001
1000/1000 [==============================] - 148s 148ms/step - loss: 1.3405 - accuracy: 0.7117 - val_loss: 1.3212 - val_accuracy: 0.6997
Epoch 4/200
Learning rate:  0.001
1000/1000 [==============================] - 148s 148ms/step - loss: 1.1310 - accuracy: 0.7477 - val_loss: 1.5094 - val_accuracy: 0.6457
Epoch 5/200
Learning rate:  0.001
1000/1000 [==============================] - 146s 146ms/step - loss: 0.9798 - accuracy: 0.7736 - val_loss: 1.2959 - val_accuracy: 0.6908
Epoch 6/200
Learning rate:  0.001
1000/1000 [==============================] - 151s 151ms/step - loss: 0.8779

1000/1000 [==============================] - 146s 146ms/step - loss: 0.3955 - accuracy: 0.9578 - val_loss: 1.5124 - val_accuracy: 0.7383
Epoch 48/200
Learning rate:  0.001
1000/1000 [==============================] - 145s 145ms/step - loss: 0.3883 - accuracy: 0.9617 - val_loss: 1.3856 - val_accuracy: 0.7485
Epoch 49/200
Learning rate:  0.001
1000/1000 [==============================] - 146s 146ms/step - loss: 0.3912 - accuracy: 0.9593 - val_loss: 1.3395 - val_accuracy: 0.7554
Epoch 50/200
Learning rate:  0.001
1000/1000 [==============================] - 146s 146ms/step - loss: 0.3798 - accuracy: 0.9612 - val_loss: 1.3192 - val_accuracy: 0.7655
Epoch 51/200
Learning rate:  0.001
1000/1000 [==============================] - 147s 147ms/step - loss: 0.3864 - accuracy: 0.9580 - val_loss: 1.3187 - val_accuracy: 0.7637
Epoch 52/200
Learning rate:  0.001
1000/1000 [==============================] - 147s 147ms/step - loss: 0.3843 - accuracy: 0.9604 - val_loss: 1.4585 - val_accuracy: 0.7467
Epo

Epoch 95/200
Learning rate:  0.0001
1000/1000 [==============================] - 147s 147ms/step - loss: 0.1588 - accuracy: 0.9997 - val_loss: 1.2746 - val_accuracy: 0.8073
Epoch 96/200
Learning rate:  0.0001
1000/1000 [==============================] - 147s 147ms/step - loss: 0.1529 - accuracy: 0.9997 - val_loss: 1.2988 - val_accuracy: 0.8091
Epoch 97/200
Learning rate:  0.0001
1000/1000 [==============================] - 147s 147ms/step - loss: 0.1478 - accuracy: 0.9995 - val_loss: 1.3465 - val_accuracy: 0.8055
Epoch 98/200
Learning rate:  0.0001
1000/1000 [==============================] - 147s 147ms/step - loss: 0.1440 - accuracy: 0.9992 - val_loss: 1.3189 - val_accuracy: 0.8042
Epoch 99/200
Learning rate:  0.0001
1000/1000 [==============================] - 147s 147ms/step - loss: 0.1397 - accuracy: 0.9993 - val_loss: 1.3502 - val_accuracy: 0.8015
Epoch 100/200
Learning rate:  0.0001
1000/1000 [==============================] - 147s 147ms/step - loss: 0.1355 - accuracy: 0.9995 - v

1000/1000 [==============================] - 147s 147ms/step - loss: 0.0791 - accuracy: 1.0000 - val_loss: 1.3682 - val_accuracy: 0.8082
Epoch 143/200
Learning rate:  1e-05
1000/1000 [==============================] - 147s 147ms/step - loss: 0.0784 - accuracy: 1.0000 - val_loss: 1.3546 - val_accuracy: 0.8087
Epoch 144/200
Learning rate:  1e-05
1000/1000 [==============================] - 147s 147ms/step - loss: 0.0778 - accuracy: 1.0000 - val_loss: 1.3610 - val_accuracy: 0.8095
Epoch 145/200
Learning rate:  1e-05
1000/1000 [==============================] - 147s 147ms/step - loss: 0.0771 - accuracy: 1.0000 - val_loss: 1.3590 - val_accuracy: 0.8093
Epoch 146/200
Learning rate:  1e-05
1000/1000 [==============================] - 147s 147ms/step - loss: 0.0762 - accuracy: 1.0000 - val_loss: 1.3605 - val_accuracy: 0.8094
Epoch 147/200
Learning rate:  1e-05
1000/1000 [==============================] - 147s 147ms/step - loss: 0.0754 - accuracy: 1.0000 - val_loss: 1.3563 - val_accuracy: 0.810

1000/1000 [==============================] - 147s 147ms/step - loss: 0.0619 - accuracy: 1.0000 - val_loss: 1.3937 - val_accuracy: 0.8090
Epoch 190/200
Learning rate:  5e-07
1000/1000 [==============================] - 147s 147ms/step - loss: 0.0619 - accuracy: 1.0000 - val_loss: 1.3923 - val_accuracy: 0.8086
Epoch 191/200
Learning rate:  5e-07
1000/1000 [==============================] - 146s 146ms/step - loss: 0.0619 - accuracy: 1.0000 - val_loss: 1.3946 - val_accuracy: 0.8085
Epoch 192/200
Learning rate:  5e-07
1000/1000 [==============================] - 146s 146ms/step - loss: 0.0618 - accuracy: 1.0000 - val_loss: 1.3927 - val_accuracy: 0.8092
Epoch 193/200
Learning rate:  5e-07
1000/1000 [==============================] - 146s 146ms/step - loss: 0.0618 - accuracy: 1.0000 - val_loss: 1.3940 - val_accuracy: 0.8088
Epoch 194/200
Learning rate:  5e-07
1000/1000 [==============================] - 147s 147ms/step - loss: 0.0617 - accuracy: 1.0000 - val_loss: 1.3966 - val_accuracy: 0.809

In [7]:
model.evaluate(x_test, y_test, batch_size=batch_size)

10000/10000 [==============================] - 10s 964us/step


[1.39825473934412, 0.808899998664856]

In [5]:
 model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                        validation_data=(x_test, y_test),
                        epochs=202, verbose=1, workers=4,
                        callbacks=callbacks, initial_epoch=201)

Epoch 202/202
Learning rate:  5e-07
1000/1000 [==============================] - 167s 167ms/step - loss: 0.0614 - accuracy: 1.0000 - val_loss: 1.3983 - val_accuracy: 0.8089


In [6]:
model.evaluate_generator(datagen.flow(x_test, y_test))

[0.2651097774505615, 0.5168690085411072]